In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Load the dataset
data = pd.read_csv("Raw_MAIN_DATASET_IMPUTATION_after_deleting_stai_badge_activity_step_goal.csv")

# Check for missing values
print("Missing values in dataset:\n", data.isnull().sum())

# Encode the target variable
label_encoder = LabelEncoder()
data['Emotions'] = label_encoder.fit_transform(data['Emotions'])

# Split the data into features and target
X = data.drop(['Emotions', 'id', 'date'], axis=1)  # Drop non-numeric features
y = data['Emotions']

# Print the original size of the dataset
print("Original dataset size:", X.shape[0])

# Apply Random Oversampling
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)

# Print the size of the resampled dataset
print("Resampled dataset size:", X_resampled.shape[0])


2024-10-04 17:16:13.771738: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-04 17:16:13.849942: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-04 17:16:13.909458: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-04 17:16:13.920558: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-04 17:16:13.992753: I tensorflow/core/platform/cpu_feature_guar

Missing values in dataset:
 id                       0
date                     0
nightly_temperature      0
nremhr                   0
rmssd                    0
                        ..
WORK/SCHOOL              0
positive_affect_score    0
negative_affect_score    0
stai_stress              0
Emotions                 0
Length: 67, dtype: int64
Original dataset size: 7434
Resampled dataset size: 48987


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Load the dataset
data = pd.read_csv("Raw_MAIN_DATASET_IMPUTATION_after_deleting_stai_badge_activity_step_goal.csv")

# Check for missing values
print("Missing values in dataset:\n", data.isnull().sum())

# Encode the target variable
label_encoder = LabelEncoder()
data['Emotions'] = label_encoder.fit_transform(data['Emotions'])

# Split the data into features and target
X = data.drop(['Emotions', 'id', 'date'], axis=1)  # Drop non-numeric features
y = data['Emotions']

# Apply Random Oversampling
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)

# Split the resampled dataset into training (64%) and temporary testing set (36%)
X_train, X_temp, y_train, y_temp = train_test_split(X_resampled, y_resampled, test_size=0.36, random_state=42)

# Split the temporary testing set into validation (16%) and testing (20%)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=20/36, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Define a simple feedforward neural network model
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))  # Output layer for the number of classes

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_data=(X_val_scaled, y_val))

# Evaluate the model on the training set
train_loss, train_accuracy = model.evaluate(X_train_scaled, y_train)
print(f'Training Accuracy: {train_accuracy:.4f}')

# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(X_val_scaled, y_val)
print(f'Validation Accuracy: {val_accuracy:.4f}')

# Evaluate the model on the testing set
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
print(f'Test Accuracy: {test_accuracy:.4f}')


Missing values in dataset:
 id                       0
date                     0
nightly_temperature      0
nremhr                   0
rmssd                    0
                        ..
WORK/SCHOOL              0
positive_affect_score    0
negative_affect_score    0
stai_stress              0
Emotions                 0
Length: 67, dtype: int64


/home/fac/krishnandu/.local/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
980/980 ━━━━━━━━━━━━━━━━━━━━ 23s 20ms/step - accuracy: 0.3379 - loss: 1.8602 - val_accuracy: 0.5919 - val_loss: 1.2114
Epoch 2/10
980/980 ━━━━━━━━━━━━━━━━━━━━ 19s 19ms/step - accuracy: 0.5089 - loss: 1.3104 - val_accuracy: 0.6536 - val_loss: 1.0280
Epoch 3/10
980/980 ━━━━━━━━━━━━━━━━━━━━ 19s 19ms/step - accuracy: 0.5807 - loss: 1.1449 - val_accuracy: 0.6892 - val_loss: 0.9122
Epoch 4/10
980/980 ━━━━━━━━━━━━━━━━━━━━ 20s 20ms/step - accuracy: 0.6100 - loss: 1.0632 - val_accuracy: 0.7029 - val_loss: 0.8568
Epoch 5/10
980/980 ━━━━━━━━━━━━━━━━━━━━ 19s 20ms/step - accuracy: 0.6274 - loss: 1.0069 - val_accuracy: 0.7282 - val_loss: 0.7987
Epoch 6/10
980/980 ━━━━━━━━━━━━━━━━━━━━ 19s 19ms/step - accuracy: 0.6412 - loss: 0.9660 - val_accuracy: 0.7368 - val_loss: 0.7624
Epoch 7/10
980/980 ━━━━━━━━━━━━━━━━━━━━ 20s 20ms/step - accuracy: 0.6595 - loss: 0.9102 - val_accuracy: 0.7566 - val_loss: 0.7292
Epoch 8/10
980/980 ━━━━━━━━━━━━━━━━━━━━ 19s 19ms/step - accuracy: 0.6773 - loss: 0.8738 - 